In [1]:
import os
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from io import BytesIO
from base64 import b64decode
import pandas as pd
from sklearn.model_selection import train_test_split
import tqdm.notebook as tq
from transformers import BertTokenizer
from common_stuff import MultiSolver, BCEWeighted
from common_stuff import device

In [2]:
def split_post(cell):
    return str(cell["text"]).split()

df = pd.read_csv("../../post2ctr_dataset.csv")
df["splitted"] = df.apply(split_post, axis=1)

max_size = 224

transform = transforms.Compose([
    transforms.Resize((max_size, max_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")

class TemplateDataset(Dataset):
    def __init__(self, df, tokenizer, transform=None):
        self.df = df
        self.transform = transform
        self.tokenizer = tokenizer
        self.max_len = 512

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        cell = self.df.iloc[idx]
        
        img = Image.open(BytesIO(b64decode(cell["photo"])))
        img = img.convert("RGB")
        img = self.transform(img)
        
        target = cell["open_photo"] / cell["view"] if cell["view"] > 0 else 0

        msg = " ".join(cell["splitted"])
        inputs = self.tokenizer.encode_plus(
            msg,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            "img" : img,
            "input_ids" : inputs["input_ids"].flatten(),
            "attention_mask" : inputs["attention_mask"].flatten(),
            "token_type_ids" : inputs["token_type_ids"].flatten(),
            "target" : target
        }

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=seed)

train_dataset = TemplateDataset(train_df, tokenizer=tokenizer, transform=transform)
test_dataset = TemplateDataset(test_df, tokenizer=tokenizer, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

/home/vivy/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
def train_model(model, train_loader, optimizer, criterion):
    running_loss = 0.0
    model.train()
    loop = tq.tqdm(enumerate(train_loader), total=len(train_loader), leave=True, colour="steelblue")
    for batch_idx, data in loop:
        images = data["img"].to(device)
        ids = data["input_ids"].to(device, dtype=torch.long)
        mask = data["attention_mask"].to(device, dtype=torch.long)
        token_ids = data["token_type_ids"].to(device, dtype=torch.long)
        targets = data["target"].to(device, dtype=torch.float).squeeze()
                
        outputs = model(ids, mask, token_ids, images).squeeze()
        loss = criterion(outputs, targets)

        running_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

    return model, running_loss / len(train_loader)

In [4]:
def eval_model(model, test_loader, criterion):
    running_loss = 0.0
    model.eval()
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader, 0):
            images = data["img"].to(device)
            ids = data["input_ids"].to(device, dtype=torch.long)
            mask = data["attention_mask"].to(device, dtype=torch.long)
            token_ids = data["token_type_ids"].to(device, dtype=torch.long)
            targets = data["target"].to(device, dtype=torch.float).squeeze()
                    
            outputs = model(ids, mask, token_ids, images).squeeze()
            loss = criterion(outputs, targets)
            running_loss += loss.item()

    return running_loss / len(test_loader)

In [8]:
model = MultiSolver(768, 768)
model = model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.BCELoss()

epochs = 5
best_loss = float("inf")
criterion = nn.BCELoss()
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model, train_loss = train_model(model, train_loader, optimizer, criterion)
    test_loss = eval_model(model, test_loader, criterion)

    print(f"Train Loss = {train_loss:.4f}, Test Loss = {test_loss:.4f}")
    if test_loss < best_loss:
        torch.save(model.state_dict(), "../../convbert_BCE.pth")

Epoch 1/5


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.1678, Test Loss = 0.1586
Epoch 2/5


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.1648, Test Loss = 0.1583
Epoch 3/5


...

Train Loss = 0.1614, Test Loss = 0.1596
Epoch 4/5


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.1542, Test Loss = 0.1598
Epoch 5/5


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.1484, Test Loss = 0.1618


In [6]:
model = MultiSolver(768, 768)
model = model.to(device)

epochs = 6
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
best_loss = float("inf")
criterion = BCEWeighted(2)

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model, train_loss = train_model(model, train_loader, optimizer, criterion)
    test_loss = eval_model(model, test_loader, criterion)

    print(f"Train Loss = {train_loss:.4f}, Test Loss = {test_loss:.4f}")
    if test_loss < best_loss:
        torch.save(model.state_dict(), "../../convbert_BCEWeighted_2.pth")

Epoch 1/6


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.2086, Test Loss = 0.1958
Epoch 2/6


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.2044, Test Loss = 0.1942
Epoch 3/6


...

  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.1911, Test Loss = 0.1970
Epoch 5/6


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.1824, Test Loss = 0.2003
Epoch 6/6


  0%|          | 0/2353 [00:00<?, ?it/s]

In [5]:
model = MultiSolver(768, 768)
model = model.to(device)

epochs = 6
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
best_loss = float("inf")
criterion = BCEWeighted(3)

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model, train_loss = train_model(model, train_loader, optimizer, criterion)
    test_loss = eval_model(model, test_loader, criterion)

    print(f"Train Loss = {train_loss:.4f}, Test Loss = {test_loss:.4f}")
    if test_loss < best_loss:
        torch.save(model.state_dict(), "../../convbert_BCEWeighted_3.pth")

Epoch 1/6


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.2348, Test Loss = 0.2216
Epoch 2/6


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.2295, Test Loss = 0.2192
Epoch 3/6


...

Train Loss = 0.2117, Test Loss = 0.2275
Epoch 5/6


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.2034, Test Loss = 0.2197
Epoch 6/6


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.1992, Test Loss = 0.2363


In [6]:
model = MultiSolver(768, 768)
model = model.to(device)

epochs = 6
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
best_loss = float("inf")
criterion = BCEWeighted(4)

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model, train_loss = train_model(model, train_loader, optimizer, criterion)
    test_loss = eval_model(model, test_loader, criterion)

    print(f"Train Loss = {train_loss:.4f}, Test Loss = {test_loss:.4f}")
    if test_loss < best_loss:
        torch.save(model.state_dict(), "../../convbert_BCEWeighted_4.pth")

Epoch 1/6


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.2673, Test Loss = 0.2486
Epoch 2/6


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.2613, Test Loss = 0.2531
Epoch 3/6


...

Train Loss = 0.2384, Test Loss = 0.2577
Epoch 5/6


  0%|          | 0/2353 [00:00<?, ?it/s]

Train Loss = 0.2301, Test Loss = 0.2585
Epoch 6/6


  0%|          | 0/2353 [00:00<?, ?it/s]

KeyboardInterrupt: 